In [1]:
# allows to import of modules
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
nb_dir
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import pandas as pd

In [4]:
import  datatset
from torch.utils.data import DataLoader

In [5]:
# DataLoader

In [6]:
TRAIN_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/train/")
df = pd.read_csv(TRAIN_PATH/"train.csv")
df_train = df[[ "file_name", "category_id"]]

In [7]:
label_counts  = df["category_id"].value_counts()

In [8]:
num_calsses = len(label_counts) + 1

In [9]:
labels_ordered = label_counts.keys().tolist()

In [10]:
num_calsses

32094

In [11]:
# df["category_id"]
# sample = df_train.groupby('category_id').apply(lambda x: x.sample(n=20, replace=True)).reset_index(drop = True)

In [12]:
# len(sample)// 64

In [13]:
#import numpy

In [14]:
ds = datatset.HerbDataSet(df_train, TRAIN_PATH,256,label="category_id")

In [15]:
# x,y = ds[0]
# x,y

In [16]:
data_loader = DataLoader(ds, batch_size=32, shuffle=True)

In [17]:
# Model

In [18]:
import model
import torchvision.models as models

In [19]:
# m = model.get_model(models.resnet34(pretrained=True), device="gpu")

In [20]:
# m.cuda()

In [21]:
import torch.nn as nn
import torch

In [22]:
base = models.resnet18(pretrained=False)

In [23]:
base.fc.in_features

512

In [24]:
base.conv1

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [25]:
head = nn.Linear(in_features=512, out_features=num_calsses)

In [26]:
base.fc = head

In [27]:
base.cuda()
2

2

In [28]:
# Tracking

In [28]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(TRAIN_PATH/"logs/run-2")

In [29]:
import metrics
import train

In [30]:
acc =  metrics.Accuracy()
manager = metrics.MetricManager([acc], writer)

In [31]:
# Train

In [32]:
criterion = nn.CrossEntropyLoss()
# criterion = nn.AdaptiveLogSoftmaxWithLoss(num_calsses, num_calsses, [50, 500, 5000])

In [33]:
# layer_lrs = [
#     {'params': base.layer1.parameters(), 'lr': 1e-4},
#     {'params': base.layer2.parameters(), 'lr': 1e-4},
#     {'params': base.layer3.parameters(), 'lr': 1e-3},
#     {'params': base.layer4.parameters(), 'lr': 1e-3},
#     {'params': base.fc.parameters(), 'lr': 1e-2}
# ]

In [34]:
optimizer = torch.optim.Adam(base.parameters())

In [35]:
model_path=TRAIN_PATH/"models/"

In [36]:
trainer = train.Training(manager, criterion, optimizer, data_loader, 3, base, model_path )

In [37]:
! export CUDA_LAUNCH_BLOCKING=1

In [38]:
trainer.train_loop()

Epoch 0/3


KeyboardInterrupt: 